In [91]:
#!pip install yfinance
#!pip install requests
#!pip install beautifulsoup4

In [70]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

#We only care about data from the last two months
end_date = datetime.now()
start_date = end_date - relativedelta(weeks=6) - relativedelta(days=1)
end_date_formatted = str(end_date).split(" ")[0]
start_date_formatted = str(start_date).split(" ")[0]

data = yf.download("BMW.DE", start=start_date_formatted, end=end_date_formatted)

[*********************100%***********************]  1 of 1 completed


In [76]:
#Construct an array of only the data we care about (yesterday, 1wk ago, 2wk ago, 3wk ago, etc up to 6wk)
target_date = end_date - relativedelta(weeks=6) - relativedelta(days=1)
target_date_formatted = str(target_date).split(" ")[0]

data.loc[target_date_formatted]

#An array to hold our closing values (from oldest to newest)
closing_values = []

while target_date <= end_date:
    target_row = data.loc[target_date_formatted]
    closing_values.append(target_row['Close'])
    target_date = target_date + relativedelta(weeks=1)
    target_date_formatted = str(target_date).split(" ")[0]
    
closing_values

[89.66000366210938,
 95.30000305175781,
 95.37999725341797,
 88.0199966430664,
 73.0999984741211,
 72.41999816894531,
 75.9000015258789]

In [90]:
#Okay.  We've proven we can get the stock prices.  Now let's prove we can grab the sentences for our sentiment score.
import requests
from bs4 import BeautifulSoup

URL = "https://finance.yahoo.com/quote/TSLA"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="quoteNewsStream-0-Stream-Proxy")
results

news_elements = results.find_all("li", class_="js-stream-content Pos(r)")
for news_element in news_elements:
    title_element = news_element.find("h3")
    subtitle_element = news_element.find("p")
    print(title_element.text)
    print(subtitle_element.text)

Electric vehicles: ‘We’ve seen a fairly steep rise’ in searches, analyst says
Jessica Caldwell, executive director of insights at Edmunds, explains how more consumers are turning to electric vehicles.
Elon Musk Has Billions, but Still Took Out Mortgages. Here's Why
Elon Musk has founded Tesla, SpaceX, and a number of other companies.  In fact, Musk reportedly took out a combined total of $61 million in mortgages in 2018 for several properties in Bel Air and the Bay Area.  This included $50 million in new mortgage debt, as well as a refinance loan that he used to repay a mortgage that he had used to purchase a property in 2017.
Dow Jones Futures: 2022's First Real Market Rally Gains Steam; Five Stocks Flash Buy Signals
The market rally continues to gain steam, with Nvidia among stocks flashing buy signals. Time to go shopping, but don't go on a buying frenzy.
Tesla Stock Vs. BYD Stock: Tesla Berlin Nears Milestone As EV Rival BYD Sets Huge Target
Tesla stock vs. BYD stock: China's BYD i

In [3]:
#There you have it!  We've proven we can get all the data we need to run this network.  All that's left is to run the sentiments
#through a network trained on https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis and add the sentiment score,
#then construct the final dataframe and train a NN on it!
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup

df = pd.read_csv("Largest automakers by market capitalization.csv")

df = df.drop(columns=['price (USD)','Rank'])
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols
df = df[cols]

sentiments = []
six_weeks_ago = []
five_weeks_ago = []
four_weeks_ago = []
three_weeks_ago = []
two_weeks_ago = []
one_week_ago = []
yesterday = []

#We only care about data from the last two months
end_date = datetime.now()
start_date = end_date - relativedelta(weeks=6) - relativedelta(days=1)
end_date_formatted = str(end_date).split(" ")[0]
start_date_formatted = str(start_date).split(" ")[0]

for symbol in df['Symbol'].values:
    print(symbol)

    stock_url = "https://finance.yahoo.com/quote/" + symbol

    page = requests.get(stock_url)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="quoteNewsStream-0-Stream-Proxy")
    
    if results is None:
        results = soup.find(id="quoteNewsStream-0-Stream-Proxy")
        sentiments.append("No data!")
    else:
        sentiment_list = []

        news_elements = results.find_all("li", class_="js-stream-content Pos(r)")
        for news_element in news_elements:
            title_element = news_element.find("h3")
            subtitle_element = news_element.find("p")
            sentiment_list.append(title_element.text)
            sentiment_list.append(subtitle_element.text)
        
        sentiments.append(sentiment_list)

    data = yf.download(symbol, start=start_date_formatted, end=end_date_formatted)
    target_date = start_date
    target_date_formatted = str(target_date).split(" ")[0]

    #An array to hold our closing values (from oldest to newest)
    closing_values = []
    
    #print(data)
    #break
        
    while target_date <= end_date:
            try:
                data.loc[target_date_formatted]
            except:
                print(f"Error on symbol {symbol} for {target_date_formatted}")
                closing_values.append('error')
                
            else:
                target_row = data.loc[target_date_formatted]
                closing_values.append(target_row['Close'])
            target_date = target_date + relativedelta(weeks=1)
            target_date_formatted = str(target_date).split(" ")[0]
            
    six_weeks_ago.append(closing_values[0])
    five_weeks_ago.append(closing_values[1])
    four_weeks_ago.append(closing_values[2])
    three_weeks_ago.append(closing_values[3])
    two_weeks_ago.append(closing_values[4])
    one_week_ago.append(closing_values[5])
    yesterday.append(closing_values[6])

df['Stock Price (6wk ago),'] = six_weeks_ago
df['Stock Price (5wk ago),'] = five_weeks_ago
df['Stock Price (4wk ago),'] = four_weeks_ago
df['Stock Price (3wk ago),'] = three_weeks_ago
df['Stock Price (2wk ago),'] = two_weeks_ago
df['Stock Price (1wk ago),'] = one_week_ago
df['Stock Price (0wk ago),'] = yesterday
df['Sentiments'] = sentiments

cols = df.columns.tolist()
last = cols[-1:]
cols = cols[0:3] + last + cols[3:11]
df = df[cols]

file_name = "FinalProjectData_" + end_date_formatted + ".csv"
df.to_csv(file_name,index=False)

TSLA
[*********************100%***********************]  1 of 1 completed
TM
[*********************100%***********************]  1 of 1 completed
VOW3.DE
[*********************100%***********************]  1 of 1 completed
002594.SZ
[*********************100%***********************]  1 of 1 completed
DAI.DE
[*********************100%***********************]  1 of 1 completed
Error on symbol DAI.DE for 2022-03-25
GM
[*********************100%***********************]  1 of 1 completed
F
[*********************100%***********************]  1 of 1 completed
BMW.DE
[*********************100%***********************]  1 of 1 completed
STLA
[*********************100%***********************]  1 of 1 completed
HMC
[*********************100%***********************]  1 of 1 completed
RIVN
[*********************100%***********************]  1 of 1 completed
601633.SS
[*********************100%***********************]  1 of 1 completed
LCID
[*********************100%***********************]  1 of 1 c

In [2]:
df = pd.read_csv(file_name)
df

,country,Name,Symbol,Sentiments,marketcap,"Stock Price (6wk ago),","Stock Price (5wk ago),","Stock Price (4wk ago),","Stock Price (3wk ago),","Stock Price (2wk ago),","Stock Price (1wk ago),","Stock Price (0wk ago),"
0,United States,Tesla,TSLA,"[""Tesla ‘is entering a new growth phase': Anal...",911205269504,904.549988,876.349976,800.770020,839.289978,838.299988,871.599976,1013.919983
1,Japan,Toyota,TM,['Japanese Firm That Moves World’s Chips Count...,274258280448,194.410004,186.539993,182.229996,176.089996,164.429993,171.740005,180.270004
2,Germany,Volkswagen,VOW3.DE,['U.S. companies pull back from business deali...,124395549741,187.279999,184.000000,176.100006,155.360001,143.779999,155.460007,151.979996
3,China,BYD,002594.SZ,No data!,107779133682,240.119995,245.410004,244.860001,245.210007,229.100006,234.899994,247.080002
4,Germany,Daimler,DAI.DE,No data!,85824249620,69.750000,76.059998,68.790001,60.209999,59.090000,64.000000,63.119999
5,United States,General Motors,GM,"['Electric vehicles: Nio reports earnings, Tes...",73682640896,50.340000,48.820000,46.349998,44.900002,41.849998,43.660000,44.349998
6,United States,Ford,F,"['Electric vehicles: Nio reports earnings, Tes...",71636033536,18.080000,17.540001,17.150000,17.600000,16.330000,16.580000,16.830000
7,Germany,BMW,BMW.DE,No data!,66793934199,92.769997,95.970001,84.989998,77.510002,72.000000,76.669998,78.470001
8,Netherlands,Stellantis,STLA,"['Electric vehicles: Nio reports earnings, Tes...",58517344256,19.100000,18.910000,18.389999,16.469999,14.950000,16.410000,15.980000
9,Japan,Honda,HMC,"['Flying Through the Business Cycle, Fed Needs...",51654283264,30.760000,31.559999,30.570000,28.830000,27.180000,28.240000,28.480000
